In [94]:
%pip install ultralytics
%pip install pybboxes
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 30.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 9.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [148]:
import os
import re
import json
import array
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO, YOLOWorld, RTDETR
from ultralytics.utils.metrics import ConfusionMatrix, Metric, ap_per_class
from ultralytics.utils.plotting import Annotator
from pybboxes import BoundingBox
from sklearn.metrics import average_precision_score, precision_recall_curve

In [81]:
dataset_path = 'FLIR_ADAS_v2/video_thermal_test/data'
images = sorted(os.listdir(dataset_path))
images_paths = [os.path.join(dataset_path, image) for image in images]

dataset_classes = {
    1: "person",
    2: "bike",
    3: "car",
    4: "motor",
    6: "bus",
    7: "train",
    8: "truck",
    10: "light",
    11: "hydrant",
    12: "sign",
    17: "dog",
    37: "skateboard",
    73: "stroller",
    77: "scooter",
    79: "other vehicle",
    80: "wrong"
}

with open('FLIR_ADAS_v2/video_thermal_test/index.json', 'r') as file:
    annotations_data = json.load(file)

frames_annotations = annotations_data['frames']

yolo_v8 = YOLO('yolov8n.pt')
yolo_world = YOLOWorld('yolov8s-world.pt')
rt_detr = RTDETR('rtdetr-l.pt')

In [113]:
allImages = False
epoch = 250
data = images_paths if allImages else images_paths[:epoch]
frames_analyzed = len(data)

yolo_v8_results = yolo_v8.predict(data)
yolo_world_results = yolo_world.predict(data)
rt_detr_results = rt_detr.predict(data)


0: 512x640 1 person, 1 airplane, 63.8ms
1: 512x640 1 person, 2 airplanes, 63.8ms
2: 512x640 1 person, 1 airplane, 63.8ms
3: 512x640 1 person, 63.8ms
4: 512x640 1 person, 63.8ms
5: 512x640 1 person, 63.8ms
6: 512x640 1 person, 63.8ms
7: 512x640 1 person, 63.8ms
8: 512x640 1 person, 1 airplane, 63.8ms
9: 512x640 1 person, 63.8ms
10: 512x640 1 person, 1 airplane, 63.8ms
11: 512x640 1 person, 63.8ms
12: 512x640 1 person, 63.8ms
13: 512x640 1 person, 63.8ms
14: 512x640 1 person, 63.8ms
15: 512x640 1 person, 63.8ms
16: 512x640 1 person, 1 airplane, 63.8ms
17: 512x640 1 person, 63.8ms
18: 512x640 1 person, 63.8ms
19: 512x640 1 person, 63.8ms
20: 512x640 1 person, 1 airplane, 63.8ms
21: 512x640 1 person, 63.8ms
22: 512x640 1 person, 1 airplane, 63.8ms
23: 512x640 (no detections), 63.8ms
24: 512x640 1 person, 1 airplane, 63.8ms
25: 512x640 1 person, 1 airplane, 63.8ms
26: 512x640 1 person, 1 airplane, 63.8ms
27: 512x640 1 person, 63.8ms
28: 512x640 1 person, 63.8ms
29: 512x640 1 person, 1 airp

In [24]:
class Annotation:
    def __init__(self, label, bbox, conf = None):
        self.label = label
        self.bbox = bbox
        self.conf = conf

def getFrameId(image_path):
    pattern = r'frame-(\w+)-(\w+).jpg'
    matchedPattern = re.search(pattern, image_path)

    if matchedPattern:
        return matchedPattern.group(2)
    else:
        return None

def getFrameAnnotations(frameId):
    frame = next((frame for frame in frames_annotations if frame['datasetFrameId'] == frameId), None)

    if frame:
        return frame['annotations']
    else:
        return None

def getFrameSize(frameId):
    frame = next((frame for frame in frames_annotations if frame['datasetFrameId'] == frameId), None)

    if frame:
        return frame['width'], frame['height']
    else:
        return None

def denormalize(x, y, w, h, image_width, image_height):
    denorm_x = x * image_width
    denorm_y = y * image_height
    denorm_w = w * image_width
    denorm_h = h * image_height
    return denorm_x, denorm_y, denorm_w, denorm_h

def normalize(x, y, w, h, image_width, image_height):
    norm_x = x / image_width
    norm_y = y / image_height
    norm_w = w / image_width
    norm_h = h / image_height
    return norm_x, norm_y, norm_w, norm_h

def yolobbox2bbox(x,y,w,h):
    x1, y1 = x-w/2, y-h/2
    x2, y2 = x+w/2, y+h/2
    return x1, y1, x2, y2

def getIdByLabel(label):
    for category_id, category_label in dataset_classes.items():
        if category_label == label:
            return category_id
    return None

In [12]:
def getExpectedAnnotationsAndClasses(frameId, logs = False):
  expected_annotations = []
  expected_classes = {}
  annotations = getFrameAnnotations(frameId)
  width, height = getFrameSize(frameId)

  print('\nExpected Annotations: ') if logs else None
  for annotation in annotations:
      label = annotation['labels'][0]

      if label in expected_classes:
          expected_classes[label] += 1
      else:
          expected_classes[label] = 1

      bbox = [int(annotation['boundingBox']['x']), int(annotation['boundingBox']['y']), int(annotation['boundingBox']['w']), int(annotation['boundingBox']['h'])]

      coco_bbox = BoundingBox.from_coco(*bbox, image_size=(width, height))
      x,y,w,h = coco_bbox.to_yolo(return_values=True)
      expected_annotations.append(Annotation(label,[x,y,w,h]))

      if logs:
        x,y,w,h = denormalize(x,y,w,h,width,height)
        print(f"> Bounding Box: x={x}, y={y}, w={w}, h={h}, Label: {label}")

  return expected_annotations, expected_classes

In [1]:
def getPredictedAnnotationsAndClasses(frameId, result, annotator, logs = False):
  preticted_annotations = []
  preticted_classes = {}
  width, height = getFrameSize(frameId)

  print('\nPreticted Annotations: ') if logs else None
  for box in result.boxes:
    xyxy = box.xyxy[0]  # get box coordinates in (left, top, right, bottom) format
    xywh = box.xywh[0]  # get box coordinates in (x, y, w, h) format
    x,y,w,h = float(xywh[0]), float(xywh[1]), float(xywh[2]), float(xywh[3])
    
    conf = float(box.conf)

    labelId = box.cls
    label = yolo_v8.names[int(labelId)]
    
    if label in preticted_classes:
          preticted_classes[label] += 1
    else:
          preticted_classes[label] = 1

    if logs:
      annotator.box_label(xyxy, label)
      print(f"> Bounding Box: x={round(x,1)}, y={round(y,1)}, w={round(w,1)}, h={round(h,1)}, Label: {label}, Conf: {conf}")

    x,y,w,h = normalize(x,y,w,h,width,height)
    preticted_annotations.append(Annotation(label,[x,y,w,h], conf))

  return preticted_annotations, preticted_classes

In [63]:
def calculateCorrectPredictionsAndClasses(frameId, predicted_annotations, expected_annotations, iou_threshold = 0.5):
  correct_predictions = 0
  correct_classes = {}
  width, height = getFrameSize(frameId)

  #tp = []
  #conf = [] 
  #pred_cls = [] 
  #target_cls = [] 

  for predicted in predicted_annotations:
    preticted_bbox = BoundingBox.from_yolo(*predicted.bbox, image_size=(width, height))

    best_expected_iou = 0
    best_expected_annotation_index = None

    for index, expected in enumerate(expected_annotations):
      if expected.label == predicted.label:
        expected_bbox = BoundingBox.from_yolo(*expected.bbox, image_size=(width, height))
        iou = preticted_bbox.iou(expected_bbox)
        if (iou > best_expected_iou):
          best_expected_iou = iou
          best_expected_annotation_index = index

    if(best_expected_iou >= iou_threshold):
      removed_annotation = expected_annotations.pop(best_expected_annotation_index)
      correct_predictions += 1

      #target_cls.append(removed_annotation.label)
      #tp.append(True)
      
      if predicted.label in correct_classes:
          correct_classes[predicted.label] += 1
      else:
          correct_classes[predicted.label] = 1
    #else:
      #tp.append(False)
      #target_cls.append("background") #!classe fittizia
      
    #conf.append(predicted.conf)
    #pred_cls.append(predicted.label)
    

  return correct_predictions, correct_classes #, tp, conf, pred_cls, target_cls

In [3]:
def analyzeResults(results, verboseLogs, frameLogs):
  metric = Metric()

  total_precision = 0
  total_recall = 0
  total_f1_score = 0

  total_class_metrics = {label: {"precisions": [], "recalls": [], "f1-scores": []} for label in dataset_classes.values()}
  
  for index, result in enumerate(results):
      frameId = getFrameId(images_paths[index])

      if verboseLogs:
        print('\n----------------------------------------------------------------------------')
        print(f'\nFrame ID: "{frameId}"')

      annotator = Annotator(cv2.imread(images_paths[index]))

      expected_annotations, expected_classes = getExpectedAnnotationsAndClasses(frameId, verboseLogs)
      preticted_annotations, preticted_classes = getPredictedAnnotationsAndClasses(frameId, result, annotator, verboseLogs)

      if verboseLogs:
        print('\n')
        cv2_imshow(annotator.result())

      total_predictions = len(preticted_annotations)
      #correct_predictions, correct_classes, tp, conf, pred_cls, target_cls = calculateCorrectPredictionsAndClasses(frameId, preticted_annotations, expected_annotations)
      #apc = ap_per_class(np.array(tp, dtype=np.bool_), np.array(conf), np.array(pred_cls), np.array(target_cls))
      correct_predictions, correct_classes = calculateCorrectPredictionsAndClasses(frameId, preticted_annotations, expected_annotations)
      wrong_predictions = total_predictions - correct_predictions
      missing_predictions = len(expected_annotations)

      if verboseLogs:
        print("\nTotal Predictions: ", total_predictions) # true positives + false positives
        print("Correct Predictions: ", correct_predictions) # true positives
        print("Wrong Predictions: ", wrong_predictions) # false positives
        print("Missing Predictions: ", missing_predictions) # false negatives

      precision = (correct_predictions / total_predictions) if total_predictions > 0 else 0
      recall = (correct_predictions / (correct_predictions + missing_predictions)) if correct_predictions + missing_predictions > 0 else 0
      f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

      if verboseLogs:
        print(f'\nPrecision: {round(precision * 100, 2)}%')
        print(f'Recall: {round(recall * 100, 2)}%')
        print(f'F1-Score: {round(f1_score * 100, 2)}%')
      elif frameLogs:
        print(f'Frame "{frameId}" | Precision: {round(precision * 100, 2)}% | Recall: {round(recall * 100, 2)}% | F1-Score: {round(f1_score * 100, 2)}%')

      total_precision += precision
      total_recall += recall
      total_f1_score += f1_score

      class_metrics = {}

      for label in expected_classes:
        class_metrics[label] = {"precision": 0, "recall": 0, "f1-score": 0}

        y_true = []
        y_score = []

        expected = expected_classes.get(label, 0)
        preticted = preticted_classes.get(label, 0)
        correct = correct_classes.get(label, 0) # TPma gli 
        wrong = preticted - correct # FP
        missing = expected - correct # FN

        class_precision = correct / (correct + wrong) if (correct + wrong) > 0 else 0
        class_recall = correct / (correct + missing) if (correct + missing) > 0 else 0
        class_f1_score = 2 * (class_precision * class_recall) / (class_precision + class_recall) if (class_precision + class_recall) > 0 else 0

        if verboseLogs:
          print(f'Class "{label}" | Precision: {round(class_precision * 100, 2)}% | Recall: {round(class_recall * 100, 2)}% | F1-Score: {round(class_f1_score * 100, 2)}%')

        class_metrics[label]['precision'] = class_precision
        class_metrics[label]['recall'] = class_recall
        class_metrics[label]['f1-score'] = class_f1_score

    

      for label in class_metrics:
        total_class_metrics[label]['precisions'].append(class_metrics[label]['precision'])
        total_class_metrics[label]['recalls'].append(class_metrics[label]['recall'])
        total_class_metrics[label]['f1-scores'].append(class_metrics[label]['f1-score'])

  print('----------------------------------------------------------------------------') if frameLogs else None

  avg_precision = total_precision / frames_analyzed
  avg_recall = total_recall / frames_analyzed
  avg_f1_score = total_f1_score / frames_analyzed

  avg_class_metrics = {}
  
  for label in total_class_metrics:
    if len(total_class_metrics[label]["precisions"]) > 0:
      avg_class_metrics[label] = {"precision": 0, "recall": 0, "f1-score": 0}
      avg_class_metrics[label]["precision"] = sum(total_class_metrics[label]["precisions"]) / len(total_class_metrics[label]["precisions"])
      avg_class_metrics[label]["recall"] = sum(total_class_metrics[label]["recalls"]) / len(total_class_metrics[label]["recalls"])
      avg_class_metrics[label]["f1-score"] = sum(total_class_metrics[label]["f1-scores"]) / len(total_class_metrics[label]["f1-scores"])

  #print(avg_class_metrics)

  return avg_precision, avg_recall, avg_f1_score, avg_class_metrics

In [154]:
def calculateMeanAveragePrecision(results, iou_threshold = 0.5):
  
  class_y_data = {label: {'y_true': [], 'y_scores': []} for label in dataset_classes.values()}
  
  for index, result in enumerate(results):
      frameId = getFrameId(images_paths[index])
      width, height = getFrameSize(frameId)
      annotations = getFrameAnnotations(frameId)

      expected_annotations = []
      for annotation in annotations:
          label = annotation['labels'][0]
          bbox = [int(annotation['boundingBox']['x']), int(annotation['boundingBox']['y']), int(annotation['boundingBox']['w']), int(annotation['boundingBox']['h'])]
          coco_bbox = BoundingBox.from_coco(*bbox, image_size=(width, height))
          x,y,w,h = coco_bbox.to_yolo(return_values=True)
          expected_annotations.append(Annotation(label,[x,y,w,h]))    
    
      predicted_annotations = []
      for box in result.boxes:
        xywh = box.xywh[0]
        x,y,w,h = float(xywh[0]), float(xywh[1]), float(xywh[2]), float(xywh[3])
        conf = float(box.conf)
        labelId = box.cls
        label = yolo_v8.names[int(labelId)]
        x,y,w,h = normalize(x,y,w,h,width,height)
        predicted_annotations.append(Annotation(label,[x,y,w,h], conf))
    
    
      for predicted in predicted_annotations:
        preticted_bbox = BoundingBox.from_yolo(*predicted.bbox, image_size=(width, height))
    
        best_expected_iou = 0
        best_expected_annotation_index = None
    
        for index, expected in enumerate(expected_annotations):
          if expected.label == predicted.label:
            expected_bbox = BoundingBox.from_yolo(*expected.bbox, image_size=(width, height))
            iou = preticted_bbox.iou(expected_bbox)
            if (iou > best_expected_iou):
              best_expected_iou = iou
              best_expected_annotation_index = index
    
        if(best_expected_iou >= iou_threshold):
          removed_annotation = expected_annotations.pop(best_expected_annotation_index)
          # aggiunge TP ad y_true della classe
          class_y_data[predicted.label]['y_true'].append(1)
          class_y_data[predicted.label]['y_scores'].append(predicted.conf)
        else:
          # aggiunge FP ad y_true della classe erroneamente detectata (se presente nel dataset, altrimenti a 'wrong')
          if label in class_y_data:
              class_y_data[label]['y_true'].append(0)
              class_y_data[label]['y_scores'].append(predicted.conf)
          else:
              class_y_data['wrong']['y_true'].append(0)
              class_y_data['wrong']['y_scores'].append(predicted.conf)

      #for expected in expected_annotations:
        # aggiunge FN ad y_true della classe
        #class_y_data[expected.label]['y_true'].append(0)
        #class_y_data[expected.label]['y_scores'].append(0)

  class_y_data_filtered = {
    label: data for label, data in class_y_data.items() 
    if any(data['y_true']) and any(data['y_scores']) and label != 'wrong'
       and not all(value == 0 for value in data['y_true'])
       and not all(value == 0 for value in data['y_scores'])
  }

  ap_values = []

  for label, data in class_y_data_filtered.items():
      y_true = data['y_true']
      y_scores = data['y_scores']

      y_true_sorted = [y_true[idx] for idx in sorted(range(len(y_scores)), key=lambda i: y_scores[i], reverse=True)]
      y_scores_sorted = sorted(y_scores, reverse=True)
      #print(label, y_true_sorted, y_scores_sorted )

      precision, recall, _ = precision_recall_curve(y_true_sorted, y_scores_sorted)
      plt.plot(recall, precision, marker='.')
      plt.xlabel('Recall')
      plt.ylabel('Precision')
      plt.title('Precision-Recall Curve for class "'+label+'"')
      plt.grid(True)
      plt.show()
    
      ap = average_precision_score(y_true, y_scores)
      print(f"AP for class '{label}': {ap:.4f}")
      ap_values.append(ap)

  mAP = sum(ap_values) / len(ap_values)
  print(f"mAP: {mAP:.4f}")

  return mAP

In [5]:
yolo_v8_mAP = calculateMeanAveragePrecision(yolo_v8_results)
yolo_world_map = calculateMeanAveragePrecision(yolo_world_results)
rt_detr_map = calculateMeanAveragePrecision(rt_detr_results)

NameError: name 'calculateMeanAveragePrecision' is not defined

In [4]:
verboseLogs = False
frameLogs = False

yolo_v8_avg_precision, yolo_v8_avg_recall, yolo_v8_avg_f1_score, yolo_v8_avg_class_metrics = analyzeResults(yolo_v8_results, verboseLogs, frameLogs)
yolo_world_avg_precision, yolo_world_avg_recall, yolo_world_avg_f1_score, yolo_world_avg_class_metrics = analyzeResults(yolo_world_results, verboseLogs, frameLogs)
rt_detr_avg_precision, rt_detr_avg_recall, rt_detr_avg_f1_score, rt_detr_avg_class_metrics = analyzeResults(rt_detr_results, verboseLogs, frameLogs)

print('\n----------------------------------------------------------------------------') if verboseLogs else None

print(f'Total Frames: {frames_analyzed}')
print(f'YOLOv8    | Precision: {round(yolo_v8_avg_precision * 100, 2)}% | Recall: {round(yolo_v8_avg_recall * 100, 2)}% | F1-Score: {round(yolo_v8_avg_f1_score * 100, 2)}%')
print(f'YOLOworld | Precision: {round(yolo_world_avg_precision * 100, 2)}% | Recall: {round(yolo_world_avg_recall * 100, 2)}% | F1-Score: {round(yolo_world_avg_f1_score * 100, 2)}%')
print(f'RT-DETR   | Precision: {round(rt_detr_avg_precision * 100, 2)}% | Recall: {round(rt_detr_avg_recall * 100, 2)}% | F1-Score: {round(rt_detr_avg_f1_score * 100, 2)}%')


NameError: name 'yolo_v8_results' is not defined

In [16]:
def getConfusionMatrix(results):
  cm = ConfusionMatrix(len(dataset_classes))

  for index, result in enumerate(results):
    frameId = getFrameId(images_paths[index])
    width, height = getFrameSize(frameId)
    annotations = getFrameAnnotations(frameId)

    print('\n----------------------------------------------------------------------------')
    print(f'\nFrame ID: "{frameId}"')

    annotator = Annotator(cv2.imread(images_paths[index]))

    expected_annotations = []
    gt_bboxes = []
    gt_cls = []
    ''' gt_bboxes = np.empty((0, 4))
    gt_cls = np.empty((0, 1)) '''

    print('\nExpected Annotations: ')
    for annotation in annotations:
      label = annotation['labels'][0]
      labelId = getIdByLabel(label)
      bbox = [int(annotation['boundingBox']['x']), int(annotation['boundingBox']['y']), int(annotation['boundingBox']['w']), int(annotation['boundingBox']['h'])]
      coco_bbox = BoundingBox.from_coco(*bbox, image_size=(width, height))
      x,y,w,h = coco_bbox.to_yolo(return_values=True)
      x,y,w,h = denormalize(x,y,w,h,width,height)
      x1,y1,x2,y2 = yolobbox2bbox(x,y,w,h)

      print(f"> Bbox: {x1,y1,x2,y2}, Label: {labelId}-{label}")
      expected_annotations.append(Annotation(label,[x1,y1,x2,y2]))
      gt_bbox = torch.tensor([x1, y1, x2, y2])
      gt_bboxes.append(gt_bbox)
      gt_cls.append(labelId)

      ''' gt_bbox = np.array([x1, y1, x2, y2])
      gt_bboxes = np.vstack((gt_bboxes, gt_bbox))
      gt_cls = np.vstack((gt_cls, label)) '''

    gt_bboxes = torch.stack(gt_bboxes)
    gt_cls = torch.tensor(gt_cls)

    preticted_annotations = []
    detections = []
    ''' detections = np.empty((0, 6)) '''

    print('\nPreticted Annotations: ')
    for box in result.boxes:
      labelId = int(box.cls) + 1
      label = str(yolo_v8.names[labelId - 1])
      conf = float(box.conf[0])
      xyxy = box.xyxy[0]
      annotator.box_label(xyxy, label)
      x1,y1,x2,y2 = float(xyxy[0]), float(xyxy[1]), float(xyxy[2]), float(xyxy[3])
      print(f"> Bbox: {x1,y1,x2,y2}, Label: {labelId}-{label}, Conf: {conf}")
      preticted_annotations.append(Annotation(label,[x1,y1,x2,y2]))
      detection = torch.tensor([x1, y1, x2, y2, conf, labelId])
      detections.append(detection)
      ''' detection = np.array([x1, y1, x2, y2, conf, label], dtype=object)
      detections = np.vstack((detections, detection)) '''

    if len(detections) == 0:
      detections = torch.empty((0, 6))
    else:
      detections = torch.stack(detections)

    '''  print(gt_bboxes)
    print(gt_cls)
    print(detections) '''

    cm.process_batch(detections, gt_bboxes, gt_cls)
    #cv2_imshow(annotator.result())



  return cm


In [17]:
#yolo_v8_cm = getConfusionMatrix(yolo_v8_results)

''' yolo_v8_cm.print()
yolo_v8_cm.plot(True,"/", ('label1','label2')) '''



' yolo_v8_cm.print()\nyolo_v8_cm.plot(True,"/", (\'label1\',\'label2\')) '